In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions
import boto3
import sys
import re
from datetime import datetime
import configparser
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1671617803236_0008,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
RawSchema=StructType([
                                    StructField("Time-In-Secs", DoubleType(), nullable=True), \
                                    StructField("Time-Per-Sec", StringType(), nullable=True),\
                                    StructField("BARO-CORRECTED-PALT-DERIVED", StringType(), nullable=True),\
                                    StructField("Altitude-BaroL", StringType(), nullable=True),\
                                    StructField("Magnetic-Heading", StringType(), nullable=True),\
                                    StructField("Pitch-Attitude", StringType(), nullable=True),\
                                    StructField("Roll-Attitude", StringType(), nullable=True),\
                                    StructField("Radio-Altitude", StringType(), nullable=True),\
                                    StructField("Nose-WOW", StringType(), nullable=True),\
                                    StructField("Main-WOW", StringType(), nullable=True),\
                                    StructField("Normal-Acceleration", StringType(), nullable=True),\
                                    StructField("Lateral-Acceleration", StringType(), nullable=True),\
                                    StructField("Longitudinal-Acceleration", StringType(), nullable=True),\
                                    StructField("Calibrated-Airspeed-L", StringType(), nullable=True),\
                                    StructField("AFCS-Selected-Speed", StringType(), nullable=True),\
                                    StructField("Vertical-Speed-Derived", StringType(), nullable=True),\
                                    StructField("Ground-Speed", StringType(), nullable=True),\
                                    StructField("Flap-Control-Selection", StringType(), nullable=True),\
                                    StructField("FLAP-POSITION", StringType(), nullable=True),\
                                    StructField("Power-Lever-Pos-L", StringType(), nullable=True),\
                                    StructField("Power-Lever-Pos-R", StringType(), nullable=True),\
                                    StructField("Torque-TQ-Eng-L", StringType(), nullable=True),\
                                    StructField("Torque-TQ-Eng-R", StringType(), nullable=True),\
                                    StructField("Condition-Lever-L", StringType(), nullable=True),\
                                    StructField("Condition-Lever-R", StringType(), nullable=True),\
                                    StructField("Prop-RPM-Np-Eng-L", StringType(), nullable=True),\
                                    StructField("Prop-RPM-Np-Eng-R", StringType(), nullable=True),\
                                    StructField("GPWS-Alert/Warning", StringType(), nullable=True),\
                                    StructField("GPWS-Glideslope-Alert", StringType(), nullable=True),\
                                    StructField("AFCS-Glideslope-Active", StringType(), nullable=True),\
                                    StructField("AFCS-LOC-Active", StringType(), nullable=True),\
                                    StructField("AFCS-VOR-Active", StringType(), nullable=True),\
                                    StructField("GLIDE-PATH-DEV-PILOT-ABOVE", StringType(), nullable=True),\
                                    StructField("GLIDE-PATH-DEV-COPILOT-ABOVE", StringType(), nullable=True),\
                                    StructField("DME-DISTANCE-1", StringType(), nullable=True),\
                                    StructField("DME-DISTANCE-2", StringType(), nullable=True),\
                                    StructField("VOR-1-SELECT-FREQ", StringType(), nullable=True),\
                                    StructField("VOR-2-SELECT-FREQ", StringType(), nullable=True),\
                                    StructField("WIND-ANGLE", StringType(), nullable=True),\
                                    StructField("WIND-SPEED", StringType(), nullable=True),\
                                    StructField("BRAKE-PRESS-LEFT", StringType(), nullable=True),\
                                    StructField("BRAKE-PRESS-RIGHT", StringType(), nullable=True),\
                                    StructField("Fuel-Flow-No-2-KG", StringType(), nullable=True),\
                                    StructField("Fuel-Flow-No-1-KG", StringType(), nullable=True),\
                                    StructField("Thrust-Rev-Low-Beta-Eng-L", StringType(), nullable=True),\
                                    StructField("Thrust-Rev-Low-Beta-Eng-R", StringType(), nullable=True),\
                                    StructField("AFCS-Autopilot-Engaged", StringType(), nullable=True),
                                    ])
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
InputS3Filepath = 's3://sjet-datamart-bucket/PrecogFiles/Q400/128 WPS/20221208/'
raw_df=spark.read.csv(InputS3Filepath,header=True,schema=RawSchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
raw_df_withfilename_1 = raw_df.withColumn("FileName", regexp_extract(
                    regexp_replace(input_file_name(), '%20', ' '), "ALL\s(.*?)\.(\w+)$", 0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
FileInfoS3Filepath = 's3://sjet-datamart-bucket/precog/precog-file-info/q400/128-wps/20221208/FileInfo_20221208.csv'
flight_data_info = spark \
                            .read \
                            .format("csv") \
                            .option("header", "true") \
                            .load(FileInfoS3Filepath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
raw_df_withfilename_1.createOrReplaceTempView("RawData_1")
flight_data_info.dropDuplicates().createOrReplaceTempView("FlightInfo")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
raw_df_withfilename_2 = spark.sql('''

                            SELECT *,
                            
                            CASE 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 1 THEN CONCAT('00000',CAST(`Time-Per-Sec` as String)) 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 2 THEN CONCAT('0000',CAST(`Time-Per-Sec` as String)) 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 3 THEN CONCAT('000',CAST(`Time-Per-Sec` as String))
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 4 THEN CONCAT('00',CAST(`Time-Per-Sec` as String)) 
                                WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 5 THEN CONCAT('0',CAST(`Time-Per-Sec` as String))
                                ELSE `Time-Per-Sec`
                            END As Time
                                                                
                            FROM RawData_1
                                                
                    ''')

raw_df_withfilename_3 = raw_df_withfilename_2.withColumn("Time-Per-Sec-1", regexp_replace(col("Time") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" ))

raw_df_withfilename_3.createOrReplaceTempView('RawData_2')

flight_data_join_df = spark.sql(f'''

                            SELECT
                            fi.TailNo,
                            fi.Departure,
                            fi.Arrival,
                            fi.FlightNo,
                            fi.FlightDate as `Date`,
                            rw.*,
                            current_timestamp() as `LoadedDate`,
                            'precog-spark' as `LoadedBy`,
                            '128 WPS' as `DataFrameName`,
                            fi.year,
                            fi.month

                            FROM RawData_2 rw

                            JOIN FlightInfo fi 
                            ON fi.FileName == rw.FileName
            
                                        ''')

flight_data_join_df.createOrReplaceTempView('flight_data_join_df')

UTC = spark.sql('''

                    SELECT TailNo, Departure, Arrival, FlightNo, Date, Filename, to_timestamp(concat(Date,' ',first_value(`Time-Per-Sec-1`,true))) AS UTC_TIMEDATE
                    
                    FROM flight_data_join_df
                    
                    Group By TailNo, Departure, Arrival, FlightNo, Date, Filename

                            ''')

UTC.createOrReplaceTempView('UTC')

UTC_TIMEDATE = spark.sql('''

                SELECT
                            
                            
                            
                            rw.*,
                            
                            fi.UTC_TIMEDATE
                            
                            FROM flight_data_join_df rw

                            JOIN UTC fi 
                            
                            ON fi.FileName = rw.FileName



                            ''')

UTC_TIMEDATE.createOrReplaceTempView('UTC_TIMEDATE')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
MainQueryDF = spark.read.csv('s3://sjet-datamart-bucket/precog/pilot-info/', header = True)

                

MainQueryDF.dropDuplicates().createOrReplaceTempView('MainQueryDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
Arms_Join_DF = spark.sql('''
                                            SELECT
                                                    AD.FlightID,
                                                    AD.DesigFlightNumber,
                                                    AD.FlightNumber AS `FlightNumberARMS`,
                                                    AD.FlightDate,
                                                    AD.FromCity,
                                                    AD.ToCity,
                                                    AD.BlockTime,
                                                    AD.ECN,
                                                    AD.EmployeeName,
                                                    AD.RANK,
                                                    CAST(AD.ChocksOff AS timestamp) AS ChocksOff,
                                                    CAST(AD.ChocksOn AS timestamp) AS ChocksOn,
                                                    AD.DOMINT,
                                                    AD.AircraftType,
                                                    AD.AircraftModel,  
                                                    AD.Payload,
                                                    RDF.*
                                                    
                                                    FROM UTC_TIMEDATE RDF
                                                    LEFT JOIN MainQueryDF AD ON RDF.FlightNo=AD.DesigFlightNumber
                                                                        AND RDF.TailNo = AD.AcRegNo
                                                                        AND RDF.Date=AD.UTCFlightDate
                                                                        AND RDF.Departure=AD.FromCity
                                                                        AND RDF.Arrival=AD.Tocity
                                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
Arms_Join_DF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+--------------------+-----+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+-----------------+----------+----------+----------------+-----------------+------------

In [7]:
raw_df_withfilename_2 = spark.sql('''

        SELECT *,
        
        CASE 
            WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 1 THEN CONCAT('00000',CAST(`Time-Per-Sec` as String)) 
            WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 2 THEN CONCAT('0000',CAST(`Time-Per-Sec` as String)) 
            WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 3 THEN CONCAT('000',CAST(`Time-Per-Sec` as String))
            WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 4 THEN CONCAT('00',CAST(`Time-Per-Sec` as String)) 
            WHEN LENGTH(CAST(`Time-Per-Sec` as String)) = 5 THEN CONCAT('0',CAST(`Time-Per-Sec` as String))
            ELSE `Time-Per-Sec`
        END As Time
                                            
        FROM RawData_1
                            
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
raw_df_withfilename_3 = raw_df_withfilename_2.withColumn("Time-Per-Sec-1", regexp_replace(col("Time") , "(\\d{2})(\\d{2})(\\d{2})" , "$1:$2:$3" ))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
raw_df_withfilename_3.createOrReplaceTempView('RawData_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
#raw_df_withfilename_3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
flight_data_info_With_J = spark.sql('''

                SELECT * FROM FlightInfo
                
                Where Filename Like ('%SEJ%J%')


''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
flight_data_info_With_J.createOrReplaceTempView('flight_data_info_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
flight_data_info_With_J.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------+-------+--------+----------+--------------------+----+--------+
|TailNo|Departure|Arrival|FlightNo|FlightDate|            FileName|year|   month|
+------+---------+-------+--------+----------+--------------------+----+--------+
|VT-SQG|      JAI|    DEL| SG 2J44|2022-12-09|ALL ac=VT-SQG dep...|2022|December|
|VT-SUR|      HJR|    DEL| SG 2J32|2022-12-08|ALL ac=VT-SUR dep...|2022|December|
|VT-SQG|      DEL|    DHM| SG 2J45|2022-12-09|ALL ac=VT-SQG dep...|2022|December|
|VT-SQA|      JAI|    AMD| SG 3J62|2022-12-08|ALL ac=VT-SQA dep...|2022|December|
|VT-SQA|      AMD|    PNQ| SG 1J77|2022-12-09|ALL ac=VT-SQA dep...|2022|December|
|VT-SQC|      DEL|    JLR| SG 2J54|2022-12-09|ALL ac=VT-SQC dep...|2022|December|
|VT-SQB|      HYD|    HYD| SG 2J96|2022-12-09|ALL ac=VT-SQB dep...|2022|December|
|VT-SUR|      DEL|    UDR| SG 2J41|2022-12-09|ALL ac=VT-SUR dep...|2022|December|
+------+---------+-------+--------+----------+--------------------+----+--------+

In [14]:
flight_data_info_Without_J = spark.sql('''
                                            SELECT * FROM FlightInfo
                                            
                                            WHERE Filename NOT LIKE ('%SEJ%J%')

''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
flight_data_info_Without_J.createOrReplaceTempView('flight_data_info_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#flight_data_info_Without_J.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
flight_data_join_df_Without_J = spark.sql(f'''

                            
                            SELECT
                            
                                fi.TailNo,

                                fi.Departure,

                                fi.Arrival,

                                fi.FlightNo,

                                fi.FlightDate as `Date`,

                                rw.*,

                                current_timestamp() as `LoadedDate`,

                                'precog-spark' as `LoadedBy`,

                                '512 WPS' as `DataFrameName`,

                                fi.year,

                                fi.month

                            FROM RawData_2 rw

                            JOIN flight_data_info_Without_J fi 
                            
                            ON fi.FileName = rw.FileName
            
                                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#flight_data_join_df_Without_J.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
flight_data_join_df_Without_J.createOrReplaceTempView('flight_data_join_df_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
flight_data_join_df_With_J = spark.sql(f'''

                            
                            SELECT
                            
                            fi.TailNo,
                            
                            fi.Departure,
                            
                            fi.Arrival,
                            
                            fi.FlightNo,
                            
                            fi.FlightDate as `Date`,
                            
                            rw.*,
                            
                            current_timestamp() as `LoadedDate`,
                            
                            'precog-spark' as `LoadedBy`,
                            
                            '512 WPS' as `DataFrameName`,
                            
                            fi.year,
                            
                            fi.month

                            FROM RawData_2 rw

                            JOIN flight_data_info_With_J fi 
                            
                            ON fi.FileName = rw.FileName
                            
                            
            
                                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#flight_data_join_df_With_J.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
flight_data_join_df_With_J.createOrReplaceTempView('flight_data_join_df_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
UTC_With_J = spark.sql('''

                    SELECT TailNo, Departure, Arrival, FlightNo, Date, Filename, to_timestamp(concat(Date,' ',first_value(`Time-Per-Sec-1`,true))) AS UTC_TIMEDATE
                    
                    FROM flight_data_join_df_With_J
                    
                    Group By TailNo, Departure, Arrival, FlightNo, Date, Filename

''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
UTC_With_J.createOrReplaceTempView('UTC_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
UTC_TIMEDATE = spark.sql('''

                SELECT
                            
                            
                            
                            rw.*,
                            
                            fi.UTC_TIMEDATE
                            
                            FROM flight_data_join_df_With_J rw

                            JOIN UTC_With_J fi 
                            
                            ON fi.FileName = rw.FileName



''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
UTC_TIMEDATE.createOrReplaceTempView('UTC_TIMEDATE')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
MainQueryDF1 = spark.read.csv('s3://sjet-datamart-bucket/precog/pilot-info/', header = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
MainQueryDF1.dropDuplicates().createOrReplaceTempView('MainQueryDF1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
MainQueryDF = spark.sql('''

                            SELECT 
                                    
                                    FlightID,
                                    
                                    DesigFlightNumber,
                                    
                                    AcRegNo
                                    
                                    ,FlightNumber
                                    
                                    ,FlightDate
                                    
                                    ,UTCFlightDate
                                    
                                    ,FromCity
                                    
                                    ,ToCity
                                    
                                    ,BlockTime
                                    
                                    ,ECN
                                    
                                    ,EmployeeName
                                    
                                    ,CAST (ChocksOff AS Timestamp)
                                    
                                    ,CAST (ChocksOn AS Timestamp)
                                    
                                    ,RANK
                                    
                                    ,DOMINT
                                    
                                    ,Payload
                                    
                                    ,AircraftType
                                    
                                    ,AircraftModel

                            FROM MainQueryDF1

''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
MainQueryDF.dropDuplicates().createOrReplaceTempView('MainQueryDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
#MainQueryDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
Arms_Join_DF_Without_J = spark.sql('''
                                            
                                            SELECT
                                                    
                                                    AD.FlightID,
                                                    
                                                    AD.DesigFlightNumber,
                                                    
                                                    AD.FlightNumber AS `FlightNumberARMS`,
                                                    
                                                    AD.FlightDate,
                                                    
                                                    AD.FromCity,
                                                    
                                                    AD.ToCity,
                                                    
                                                    AD.BlockTime,
                                                    
                                                    AD.ECN,
                                                    
                                                    AD.EmployeeName,
                                                    
                                                    AD.RANK,
                                                    
                                                    CAST(AD.ChocksOff AS timestamp) AS ChocksOff,
                                                    
                                                    CAST(AD.ChocksOn AS timestamp) AS ChocksOn,
                                                    
                                                    AD.DOMINT,
                                                    
                                                    AD.AircraftType,
                                                    
                                                    AD.AircraftModel,  
                                                    
                                                    AD.Payload,
                                                    
                                                    RDF.*
                                                    
                                                    FROM flight_data_join_df_Without_J RDF
                                                    
                                                    LEFT JOIN MainQueryDF AD ON RDF.FlightNo=AD.DesigFlightNumber
                                                    
                                                    AND RDF.TailNo = AD.AcRegNo
                                                    
                                                    AND RDF.Date=AD.UTCFlightDate
                                                    
                                                    AND RDF.Departure=AD.FromCity
                                                    
                                                    AND RDF.Arrival=AD.Tocity
                                                    
                                                    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
Arms_Join_DF_Without_J.createOrReplaceTempView('Arms_Join_DF_Without_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [94]:
Arms_Join_DF_With_J_1 = spark.sql('''
                                           SELECT
                                                    
                                                    AD.FlightID,
                                                    
                                                    AD.DesigFlightNumber,
                                                    
                                                    AD.FlightNumber AS `FlightNumberARMS`,
                                                    
                                                    AD.FlightDate,
                                                    
                                                    AD.FromCity,
                                                    
                                                    AD.ToCity,
                                                    
                                                    AD.BlockTime,
                                                    
                                                    AD.ECN,
                                                    
                                                    AD.EmployeeName,
                                                    
                                                    AD.RANK,
                                                    
                                                    CAST(AD.ChocksOff AS timestamp) AS ChocksOff,
                                                    
                                                    CAST(AD.ChocksOn AS timestamp) AS ChocksOn,
                                                    
                                                    AD.DOMINT,
                                                    
                                                    AD.AircraftType,
                                                    
                                                    AD.AircraftModel,  
                                                    
                                                    AD.Payload,
                                                    
                                                    RDF.*
                                                    
                                                    FROM UTC_TIMEDATE RDF
                                                    
                                                    LEFT JOIN MainQueryDF AD ON 
                                                    
                                                    RDF.TailNo = AD.AcRegNo
                                                    
                                                    AND RDF.Date = AD.UTCFlightDate
                                                    
                                                    AND RDF.UTC_TIMEDATE BETWEEN (AD.ChocksOff - INTERVAL 345 MINUTES) AND (AD.ChocksOff - INTERVAL 315 MINUTES)
                                                    
                                                    AND RDF.Departure=AD.FromCity
                                                    
                                                    AND RDF.Arrival=AD.Tocity
                                                                        
                                                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
Arms_Join_DF_With_J_1.createOrReplaceTempView('Arms_Join_DF_With_J_1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
#Arms_Join_DF_With_J_1.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
Arms_Join_DF_With_J = spark.sql('''
                SELECT 
                    FlightID, 
                     DesigFlightNumber, 
                     FlightNumberARMS, 
                     FlightDate, 
                     FromCity, 
                     ToCity, 
                     BlockTime, 
                     ECN, 
                     EmployeeName, 
                     RANK, 
                     ChocksOff, 
                     ChocksOn, 
                     DOMINT, 
                     AircraftType, 
                     AircraftModel, 
                     Payload, 
                     TailNo, 
                     Departure, 
                     Arrival, 
                     FlightNo, 
                     Date, 
                     `Time-In-Secs`,
                     `Time-Per-Sec`, 
                     `BARO-CORRECTED-PALT-DERIVED`, 
                     `Altitude-BaroL`, 
                     `HEADING-MAGNETIC`, 
                     `PITCH-ANGLE`, 
                     `ROLL-ANGLE`, 
                     `RAD-ALT`, 
                     `WOW-NLG`, 
                     `WOW-MLG`, 
                     `ACCN-NORMAL`, 
                     `ACCN-LATERAL`, 
                     `ACCN-LONGITUDINAL`, 
                     `AIRSPEED-CAS-L`, 
                     `AFCS-SEL-AIRSPEED`, 
                     `VERTICAL-SPEED-Derived`, 
                     `FMS-GROUNDSPEED`, 
                     `FLAP-CONTROL-POS-Gated`, 
                     `FLAP-POSITION`, 
                     `PLA-1`, 
                     `PLA-2`, 
                     `TORQUE-1`, 
                     `TORQUE-2`, 
                     `CLA-1`, 
                     `CLA-2`, 
                     `NP-1`, 
                     `NP-2`, 
                     `EGPWS-ALERT-WARNING`, 
                     `EGPWS-GS-ALERT`, 
                     `AFCS-GS-MODE`, 
                     `AFCS-LOC-MODE`, 
                     `AFCS-VOR-MODE`, 
                     `ILS-GS-DEV-1`, 
                     `ILS-GS-DEV-2`, 
                     `FUEL-FLOW-2-kg-h`, 
                     `FUEL-FLOW-1-kg-h`, 
                     `DME-DISTANCE-1`, 
                     `DME-DISTANCE-2`, 
                     `VOR-FREQ-1`, 
                     `VOR-FREQ-2`, 
                     `FMS-WIND-ANGLE`, 
                     `FMS-WIND-SPEED`, 
                     `BRAKE-PRESS-LEFT`, 
                     `BRAKE-PRESS-RIGHT`, 
                     `PROP-BETA-RANGE-1`, 
                     `PROP-BETA-RANGE-2`, 
                     `AP-ENGAGED`, 
                     FileName, 
                     Time, 
                     `Time-Per-Sec-1`, 
                     LoadedDate,
                     LoadedBy, 
                     DataFrameName, 
                     year, 
                     month

                    FROM Arms_Join_DF_With_J_1

''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
#spark.sql("""SELECT current_timestamp() - INTERVAL 1 MINUTE, current_timestamp() + INTERVAL 1 MINUTE""").show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
Arms_Join_DF_With_J.createOrReplaceTempView('Arms_Join_DF_With_J')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
Arms_Join_DF_With_J.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+-------------------+-------------------+------+------------+-------------+-------+------+---------+-------+--------+----------+------------+------------+---------------------------+--------------+----------------+-----------+----------+-------+----------+----------+-----------+------------+-----------------+--------------+-----------------+----------------------+---------------+----------------------+-------------+-----+-----+--------+--------+-----+-----+----+----+-------------------+--------------+--------------+--------------+--------------+------------+------------+----------------+----------------+--------------+--------------+----------+----------+--------------+--------------+----------------+-----------------+-----------------+-----------------+--------------+--------------------+------+--------------+--------------------+------------+-------------+----+--------+
|Fli

In [109]:
Arms_Join_DF = spark.sql('''

                SELECT * FROM Arms_Join_DF_Without_J
                
                UNION
                 
                SELECT * FROM Arms_Join_DF_With_J
                

''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
Arms_Join_DF.createOrReplaceTempView('Arms_Join_DF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
spark.sql('''

                SELECT Distinct filename
                
                FROM Arms_Join_DF


''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

48